# Earth engine API to get time series

### Usual imports

In [374]:
import ee
ee.Initialize()
#import time
from datetime import datetime
#import numpy as np
#import matplotlib.pyplot as plt
import pandas as pd
import geopandas
import sys
sys.path.insert(1, 'D:\\PhD Info\\GitHub repositories\\My_Sentinel_Earth_Engine\\Earth_Engine_scripts')
import EE_library
#import matplotlib.dates as mdates

In [375]:
import folium
# Define the URL format used for Earth Engine generated map tiles.
EE_TILES = 'https://earthengine.googleapis.com/map/{mapid}/{{z}}/{{x}}/{{y}}?token={token}'
print('Folium version: ' + folium.__version__)

Folium version: 0.9.1


### Variables 

In [376]:
save_ts = "No"    # whether or not save the time series 
path_out_img = "" # Not active
# Needed later on
stats = 'mean'    #   median, max, min, minMax
out_path=""
dicc_of_geometries1={}

### Select the parameters to filter and create the image collection

In [377]:
#----------------------------Parameters: DATES, DESCENDING OR DESCENDING, POLARISATION------- 
str1='2016-12-01' # Initial date to filter the image collection from EE
date1 = ee.Date(str1)
date2=ee.Date(datetime.today()) 
#str2='2019-08-05' #  Final date to filter the image collection from EE
#date2 = ee.Date(str2) # uncomment if is not 
Direction='ASCENDING' # Sentinel 1 pass direction
orbit_No=18
#Direction='DESCENDING'
ds = date2.difference(date1, 'day')

#  Filter the image collection by the parameters previously defined

Run the following cell, click on the map, copy and paste the coordinates in the cel bellow. 

Alternatively, use a marker in this [website](http://geojson.io/#map=2/20.0/0.0) to get the coordinates of a desired location and paste them below:

In [378]:
m = folium.Map(location=[48.0194, -360.4307],tiles='Stamen Terrain',zoom_start=5)
m.add_child(folium.LatLngPopup())

In [379]:
# Otro
#Latitude=4.6018
#Longitude = -434.0945

# Barranquilla, Colombia
#Latitude =10.88725377263463
#Longitude =-74.410400390625
#Stirling
#Latitude =56.12039058281017
#Longitude =-3.9368820190429683
# Bogota
#Latitude=4.6018
#Longitude = -434.0945
# Doñana
Latitude =37.0969
Longitude=-366.1578
#sentinel1 = ee.ImageCollection('COPERNICUS/S1_GRD').filterDate(date1, date2).filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')).filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH')).filter(ee.Filter.eq('instrumentMode', 'IW')).filter(ee.Filter.eq('orbitProperties_pass', Direction)).filter(ee.Filter.eq('relativeOrbitNumber_start', orbit_No)); 
sentinel1 = ee.ImageCollection('COPERNICUS/S1_GRD').filterDate(date1, date2).filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')).filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH')).filter(ee.Filter.eq('instrumentMode', 'IW')).filterBounds(ee.Geometry.Point(Longitude,Latitude)); 

# Show a reduced image of the collection by using the ee.reduce.median() 

### Function to display a map using folium

In [380]:
#@title Mapdisplay: Display GEE objects using folium.
def Mapdisplay(center, dicc, Tiles="OpensTreetMap",zoom_start=10):
    '''
    :param center: Center of the map (Latitude and Longitude).
    :param dicc: Earth Engine Geometries or Tiles dictionary
    :param Tiles: Mapbox Bright,Mapbox Control Room,Stamen Terrain,Stamen Toner,stamenwatercolor,cartodbpositron.
    :zoom_start: Initial zoom level for the map.
    :return: A folium.Map object.
    '''
    mapViz = folium.Map(location=center,tiles=Tiles, zoom_start=zoom_start)
    for k,v in dicc.items():
      if ee.image.Image in [type(x) for x in v.values()]:
        folium.TileLayer(
            tiles = EE_TILES.format(**v),
            attr  = 'Google Earth Engine',
            overlay =True,
            name  = k
          ).add_to(mapViz)
      else:
        folium.GeoJson(
        data = v,
        name = k
          ).add_to(mapViz)
    mapViz.add_child(folium.LayerControl())
    return mapViz

### Reduce the image collection and create and additional band corresponding to the backscatter ratio

In [381]:
# Compute the median in each band, each pixel. Band names are VH_median, VV_median
Sentinel1_median=sentinel1.reduce(ee.Reducer.median())
# Compute the ratio (difference in dB)
Ratio=Sentinel1_median.select('VH_median').subtract(Sentinel1_median.select('VV_median'))
Sentinel1_median_RGB=Sentinel1_median.addBands(Ratio)

In [382]:
# The output is an Image.  Add it to the map with the following visualization parameters:
vis_param = {'min': -20, 'max':0, 'bands': ['VV_median', 'VH_median', 'VH_median_1'],'gamma': 1.6}
#vis_param = {'bands': ['VH_median', 'VV_median', 'VH_median_1'], 'gamma': 1.6}

### Display the reduced map

In [383]:
# This may take some time Depends on the size of the Image collection
median_tk = Sentinel1_median_RGB.getMapId(vis_param)

In [384]:
#center = [-8.520166448964439,-78.79737094642212]
#center = [Latitude,Longitude]
m=Mapdisplay(center,{'Sentinel 1':median_tk},zoom_start=11)
m.add_child(folium.LatLngPopup())

### Reduce the image collection by getting the standard deviation 

In [385]:
Sentinel1_std_dev=sentinel1.reduce(ee.Reducer.stdDev())
vis_param1 = {'min': 0, 'max':5, 'bands': ['VV_stdDev', 'VH_stdDev', 'angle_stdDev'],'gamma': 1}
median_tk1 = Sentinel1_std_dev.getMapId(vis_param1)
#m1=Mapdisplay(center,{'Sentinel 1 - median':median_tk},zoom_start=11)
m2=Mapdisplay(center,{'Sentinel 1 - Std_Dev':median_tk1},zoom_start=11)
m2.add_child(folium.LatLngPopup())

In [386]:
# Esparrago
#Lat1=-8.5249
#long1=-438.7914
#Lat2=-8.5227
#long2=-438.7894
#Lat3=-8.5232
##long3=-438.7887
#Lat4=-8.5235
#long4=-438.7919
#Pimientos
#Lat1=-8.4841
#long1=-438.8196
#Lat2=-8.4840
#long2=-438.8180
#Lat3=-8.4846
#long3=-438.8178
#Lat4=-8.4848
#long4=-438.8194
#Doñana Parcel A
Lat1=37.0603
long1=-366.1100
Lat2=37.0615
long2= -366.1025
Lat3=37.0608
long3=-366.1010
Lat4=37.0597
long4=-366.1093

### Create an earth engine geometry

In [387]:
Geo1 = ee.Geometry.Polygon([[      [long1, Lat1],
                                   [long2, Lat2],
                                   [long3, Lat3],
                                   [long4,  Lat4]]])


### Generate the time series of the created geometry

In [388]:
#sentinel1=sentinel1.filterBounds(Geo1)
stats = 'mean'    #   median, max, min, minMax
title='Custom geometry'
# Use automatic_ax_lim="No" if it is desired to do interparcel visual comparisons. All plot will have then same y limits               
df_region=EE_library.Time_series_of_a_region(sentinel1,Geo1,stats,title,automatic_ax_lim="Yes")

# Read shapefile

In [389]:
geo_df = geopandas.read_file("D:\\PhD Info\\EO4 Cultivar\\Ground data\\20180504_Danper_Ground_data\Shapefiles\\Danper_Campositan_parcels_boundaries_lotes_EPSG 4326 - WGS 84.shp")

Select the polygons to extract the time series:

Select the columns by which to filter the geodataframe to select the desired polygons

Specify the poligons mathing the ones in the geo_df

In [390]:
filter_geo_df_by="ID_MT"
#List_of_interesting_modulos = ["M14","M14","M14","M14","M24","M12"]
#List_of_interesting_turnos =  ["T02","T04","T05","T06","T02","T05"]
# MT with phenology
#List_of_interesting_modulos = ["M19","M11","M24","M10","M23","M06","M22","M12","M14",'M07']
#List_of_interesting_turnos =  ["T04","T01","T02","T05","T05","T02","T03","T02","T05",'T04'] # Turno representativo de ese modulo
List_of_interesting_modulos = ["M14","M24"]
List_of_interesting_turnos =  ["T05","T01"]

In [392]:
%matplotlib qt
sentinel1 = ee.ImageCollection('COPERNICUS/S1_GRD').filterDate(date1, date2).filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')).filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH')).filter(ee.Filter.eq('instrumentMode', 'IW')) 

for o in range(len(List_of_interesting_modulos)):
    #ID_MT="M23T05"
    ID_MT=List_of_interesting_modulos[o]+List_of_interesting_turnos[o]
    geo_df_IDMT=geo_df.loc[geo_df[filter_geo_df_by] == ID_MT]  
    # Use automatic_ax_lim="No" if it is desired to do interparcel visual comparisons. All plot will have then same y limits               
    dicc_of_geometries1[ID_MT]=EE_library.time_series_from_shp(sentinel1,geo_df_IDMT,ID_MT,stats,save_ts,out_path,automatic_ax_lim="No")
    #dicc_of_geometries1[ID_MT].to_csv(path_or_buf="C:\\Users\\crisj\\Box\\Danper_Data_analysis\\Output_xlsx\\EE_"+ID_MT+"_"+Direction+"_"+str(orbit_No))


In [73]:
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual

### Keyword (x=?) argument - Widget
True or False - Checkbox
'Hi there' - Text
value or (min,max) or (min,max,step) if integers are passed - IntSlider

value or (min,max) or (min,max,step) if floats are passed - FloatSlider

['orange','apple'] or `[(‘one’, 1), (‘two’, 2)] - Dropdown


In [108]:
def f(x):
    return x

In [92]:
interact(f, x=10);

interactive(children=(IntSlider(value=10, description='x', max=30, min=-10), Output()), _dom_classes=('widget-…

In [93]:
interact(f, x=True);

interactive(children=(Checkbox(value=True, description='x'), Output()), _dom_classes=('widget-interact',))

In [112]:
print("Type the direction:")
interact(f, x='Hi there!');

Type the direction:


interactive(children=(Text(value='Hi there!', description='x'), Output()), _dom_classes=('widget-interact',))

In [68]:
interact(f, x=[('one', 10), ('two', 20)]);

interactive(children=(Dropdown(description='x', options=(('one', 10), ('two', 20)), value=10), Output()), _dom…

In [161]:
from IPython.display import display
def f(a, b, c):
    display(a + b + c)
    a1=a
    b1=b
    return (a+b)

In [162]:
w = interactive(f, a=10, b=20, c=2)

In [166]:
w

interactive(children=(IntSlider(value=10, description='a', max=30, min=-10), IntSlider(value=20, description='…

In [168]:
print(w.kwargs)
#w.result[1]

{'a': 10, 'b': 20, 'c': 2}
